# ANZ Data Tutorial Notebook

This notebook will introduce some of the key ANZ tools available during the training event.The event will have functional shared services which expose simulated ANZ data services and APIs which can be used in the development of products and services. These are GCP and Data services.

The ANZ CDE and GCP is being developed so that ANZ teams can come up with novel ways to slice and present transactional data. 
The CDE is a cloud-based analytical platform which provides the capability to acquire real-time and batch third-party data for data integration, enrichment and storage. 
For the purposes of the Innovation Challenge a data Synthesizer has been developed to create batch and real-time data.

Customer Data Ecosystem (CDE) use cases are:
1. Use a data discovery environment that will contain synthetic payments and accounts
2. Explore the data and develop novel insights
3. Deploy steaming data to Data Factory
4. Surface ingested, streamed data into a mock mobile application

CDE represents both a "shared" or "master" project which curates and manages
access to data for a Party Data Factory (PDF). A PDF is a "team" project in the
Innovation challenge. A PDF contains data and services which are specific to 
a business stream. In the Innovation Challenge CDE data master is called
`shared services` and the Party Data Factory (PDF) is called a `team`.

The Shared Services project contains a GKE which runs a python program to 
simulate customer transactions - datasynth. This program generates batch files
of transactions and writes to a GKE hosted kafka topic to real-time transactions.

Each team project also contains a GKE and a jupyterhub service for data discovery.
These projects also contain a kafka and BigQuery service which can be written to.

# Quickstart (TL;DR)

## Shared Resources
- Kafka Transaction topic: `anz-uc-ss-kafka.lab.kasna.internal:9094, synth-topic`
- Kafka Payment topic: `anz-uc-ss-kafka.lab.kasna.internal:9094, payment-topic`
- Storage bucket: `gs://anz-uc-ss-storage`
- BigQuery transaction table: `anz-uc-ss.sample.transactions`
- BigQuery customers table: `anz-uc-ss.sample.customers`
- BigQuery accounts table: `anz-uc-ss.sample.accounts`

## Team Resources
- [Juypterhub login](https://anz-uc-team-1.lab.kasna.cloud)
- Storage: `gs://anz-uc-team-1-storage/`
- Team Repo: `gcloud source repos clone  --project=anz-uc-team-1`
- BigQuery dataset: `anz-uc-team-1.team_dataset`
- Kafka for team: `anz-uc-team-1.lab.kasna.internal:9094`
- [Airflow](https://anz-uc-team-1-airflow.lab.kasna.cloud)
- GKE creds: `gcloud container clusters get-credentials --project="anz-uc-team-1" --zone="australia-southeast1-a" "anz-uc-team-1-gke"`

# Jupyter

This is a tutorial on how to use datalab for the ANZ Innovation Challenge. Skip this section if you are already familiar with Jupyter. Detailed examples on how to use Jupyter and Python can be found online. 

To run python code, either press run on the toolbar or press shift+enter while a cell is selected.

In [ ]:
print('Hello World!')

There are some datasources provided in the Innovation Challenge, including Bigquery and Kafka. The following variables are provided for further use in the notebook. These are SHARED SERVICES resources which all teams will be using:

In [ ]:
KAFKA_HOST = 'anz-uc-ss-kafka.lab.kasna.internal'
KAFKA_PORT = '9094'
KAFKA = KAFKA_HOST + ":" + KAFKA_PORT
KAFKA_PUSH_TOPIC = 'synth-topic'
KAFKA_SUBSCRIBE_TOPIC = 'payment-topic'
BQ_PROJECT = 'anz-uc-ss'
BQ_DATASET = 'sample'
BQ_TABLE = 'transacions'

Your team will use resource which you can read and write to and are only for your team. These resources are:

In [ ]:
import os
PROJECT_ID = os.environ.get('PROJECT_ID')
USER = os.environ.get('JUPYTERHUB_USER')
print("Your username is: " + USER)
print("Your Project ID: " + PROJECT_ID)
TEAM_KAFKA_HOST = PROJECT_ID + '-kafka.lab.kasna.internal'
TEAM_KAFKA_PORT = '9094'
TEAM_KAFKA = TEAM_KAFKA_HOST + ":" + KAFKA_PORT
TEAM_KAFKA_PUSH_TOPIC = 'synth-topic'
TEAM_KAFKA_SUBSCRIBE_TOPIC = 'payment-topic'
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'team_dataset'
REPO = "https://source.cloud.google.com/" + PROJECT_ID + "/" + PROJECT_ID + "-team"

# BigQuery

Google Cloud Platform's BigQery enables interactive analysis of massive datasets. BigQuery uses the SQL language to query datasets and tables of data.

For the Innovation Challege there is an instance of BigQuery within the Shared Service project and within your team's project. The Shared BigQuery contains a read-only copy of the transactions generated by Datasynth.


## Explore Sample Data

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
import matplotlib
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt

The following code takes a SQL statement and queries BigQuery. The result is then saved to a Pandas Dataframe called total_births, located after the %%bigquery statement.

In [ ]:
%%bigquery total_births
SELECT
    source_year AS year,
    COUNT(is_male) AS birth_count
FROM `bigquery-public-data.samples.natality`
GROUP BY year
ORDER BY year DESC
LIMIT 15

In [ ]:
total_births.plot(x='year',y='birth_count')

## Explore Datasynth transactions

Now we'll use the Sample data, and see if we can gain any insights from it. 

We can get the base data from the shared service project bigquery, and even do some simple models via BigqueryMl on 90% of the dataset.:

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `team_dataset.simple_model`
OPTIONS(model_type='linear_reg',input_label_cols=['txn_amount_int']) AS
SELECT
CAST(txn_amount AS NUMERIC) as txn_amount_int,
CAST(txn_available_balance AS NUMERIC) as balance
FROM `anz-uc-team-1.team_dataset.transactions*`
WHERE MOD(ABS(FARM_FINGERPRINT(txn_id)),10) != 0 AND txn_date BETWEEN '2019-07-03' AND '2019-07-10'

In [ ]:
%%bigquery 
SELECT * FROM ML.EVALUATE(MODEL `team_dataset.simple_model`,
(SELECT
CAST(txn_amount AS NUMERIC) as txn_amount_int,
CAST(txn_available_balance AS NUMERIC) as balance
FROM `anz-uc-team-1.team_dataset.transactions*`
WHERE MOD(ABS(FARM_FINGERPRINT(txn_id)),10) = 0))

In [ ]:
%%bigquery predictions
SELECT * FROM ML.PREDICT(MODEL `team_dataset.simple_model`,
(SELECT
CAST(txn_amount AS NUMERIC) as txn_amount_int,
CAST(txn_available_balance AS NUMERIC) as balance
FROM `anz-uc-team-1.team_dataset.transactions*`
WHERE txn_date BETWEEN '2019-07-03' AND '2019-07-10'))

In [ ]:
h =predictions.plot('balance',y=['txn_amount_int','predicted_txn_amount_int'],style=['bo','r-'],ms=1)


Can you see any correlation between how much people have and how much people spend? Can you think of a reason for this?

# Kafka

Kafka is a stream processing platform used for high-volume publish and subscribe. CDE uses kafka to publish data to Party Data Factories. In the ANZ Innovation Challenge kafka is used to transfer data messages between the Datasynth service and the teams. The examples below walk through some of this functionality.

In [ ]:
!pip install kafka-python
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import datetime
from kafka import KafkaConsumer


## Read from Kafka synth-topic

Quick python to read off of the Shared Kafka synth-topic queue:

In [ ]:
from IPython import display
import time
print("Pull from host, " + KAFKA + " - " + KAFKA_PUSH_TOPIC)
consumer = KafkaConsumer(KAFKA_PUSH_TOPIC,auto_offset_reset='smallest',bootstrap_servers=[KAFKA])
num_records_to_show=3
i =0
for message in consumer:
    datalist = message.value.strip().decode('utf-8').split(',')
    print("===== Message " + str(i) + " =====")
    i=i+1
    for x in range(len(datalist)):
        print(datalist[x])
    if i==num_records_to_show:
        break

## Plot kafka transactions

We can also add interactively updating plots in Jupyter to use real time Kafka data about current transactions.

In [ ]:
from IPython import display
import time, json
print("Pull from host, " + KAFKA + " - " + KAFKA_PUSH_TOPIC)
consumer = KafkaConsumer(KAFKA_PUSH_TOPIC,auto_offset_reset='earliest',bootstrap_servers=[KAFKA])
num_records_to_show=300
i =0
transaction_amounts = []
for message in consumer:
  data = json.loads(message.value.strip().decode('utf-8'))
  transaction_amounts.append(float(data['txn_amount']))
  i+=1
  if i % 10==0:
    %matplotlib inline
    plt.plot(range(i),transaction_amounts,'r.',markersize=4)
    plt.ylim(0,num_records_to_show)
    plt.xlim(0,300)
    plt.gcf().set_size_inches(30,6)
    display.display(plt.gcf())
    display.clear_output(wait=True)
    plt.gcf().show()
  if i% num_records_to_show==0:
    break


# Team Repo

A team git repo has been created which provides some tools and examples for use in development. This repo is located in Google Cloud Source Repository within your project and is named: {{ PROJECT_ID }}-team

There are several methods to clone this repo and begin using the examples.

## Clone using Cloud Source Repository

To clone using CSR, navigate to the Cloud Source Repository and follow the instructions for cloning using:
- VS Code
- Gcloud SDK
- Manually using GCP credentials

In [ ]:
print("Your team repo is here: " + REPO)

## Use the jupyterlab Repo

You can interact with the team repo through the file navigator and git extenstion. To find the repo select the folder icon in the top left and select `team-repo`.

To pull/push and commit changes to the repo, you can use the git extension found in the left hand navigation. Use this tool to pull and push changes, stage files and commit. Ensure to push changes after commiting.

## Clone in Theia IDE

Eclipse Theia is a web-based IDE modeled on VS Code. An instance of Theia is available in this jupyterhub environment and can use the team repo is already availble for use. To develop in this IDE do the following:
1. Navigate to the Launcher - if this is not open you can create a new Launcher using `File -> New Launcher`
2. Select the Theia icon
3. Once Theia is loaded select `File -> Open Workspace`
4. Navigate to `team-repo` and select open.

Save the workspace to use during your next session.


# Airflow

## Airflow UI Access

Every member of a team has access to a pre-created airflow instance and can log into the airflow web-ui located at https://${PROJECT_ID}-airflow.lab.kasna.cloud.

In [ ]:
print("Your airflow instance is here:  https://" + PROJECT_ID + "-airflow.lab.kasna.cloud")

The website is authenticated by google oauth credentials and uses should login using their kasna google credentials.

![Airflow UI](team-repo/airflow/images/airflow-ui.png)

### Airflow Jobs

Once users login to the web-ui there are some pre-defined sample jobs which are defined using job templates. Airflow has been pre-configured to periodically load the jobs(dags) uploaded to a google cloud storage bucket(gs://${PROJECT_ID}-storage). 
The [cloudbuild.yaml](./cloudbuild.yaml) defines a cloud build job to upload the src directory containing job definitions to the storage bucket.

![Airflow Storage](team-repo/airflow/images/dags-storage.png)



## Yaml to DAGs 

By Convention airflow jobs are written in python however to simulate CDE, we will be utilizing an existing library to dynamically create Directed acyclic graphs (DAGs) from yaml files.

For example, the yaml file below is a simple job creating two bash operator tasks, scheduled to run hourly, and with task_2 dependant on task_1.

```
1_example_dag:
  default_args:
    start_date: 2019-09-04
    timezone: 'Australia/Melbourne' 
  schedule_interval: '0 * * * *'
  description: 'this is example dag'
  tasks:
    task_1:
      operator: airflow.operators.bash_operator.BashOperator
      bash_command: 'echo 1'
    task_2:
      operator: airflow.operators.bash_operator.BashOperator
      bash_command: 'echo 2'
      dependencies: [task_1]
```

The dag.py file in the src will render dags from .yaml files in the templates directory. One can also customize timezone, schedule_interval and other airflow job options.


## Sample Jobs

These are few sample jobs to understand airflow. 

### 1_example_dag

This is a simple job with three `bash operator` tasks to demonstrate airflow as a scheduler.

template:
[JOB1](./team-repo/airflow/src/templates/job1.yaml) 

1) Run the job. Click on the button as show in the image below.

![JOB1](team-repo/airflow/images/job1-run.png "JOB 1")

2) View the graph . Click on the link of the job and click on Graph View

![JOB1](team-repo/airflow/images/job1.png "JOB 1")

3) View the Task Log

Click on any node in the graph and select View Log.

### 2_bq_dataset_creation

This job utilizes the `gcloud operator` to create a BigQuery dataset and a table.

template:
[JOB2](./team-repo/airflow/src/templates/job2.yaml)

1) Run the job just as in Job1

2) View the graph. Click on the link of the job and click on Graph View.

![JOB2](team-repo/airflow/images/job2.png "JOB 2")

3) View the Task Log

Click on any node in the graph and select `View Log`.

4) Check Output

After the job is successful we should be able to see a dataset and a table in BigQuery console.

![JOB2 OUTPUT](team-repo/airflow/images/job2_output.png "JOB 2 OUTPUT")


### 3_bq_examples

In this job we will utilize the above created dataset and table to explore more tasks using `gcloud operators`. we will perform four task in this job. 

The first task will run a sql query to query a public dataset and create a dump the data into the table we created in the earlier job. we run the sql query from a file uploaded to the cloud storage from the src/sql directory.
.
The second task will transform the BigQuery table from first task to a cloud storage bucket in avro format.

The third task will load the data from second task from cloud storage to another BigQuery table using auto schema.

The fourth task will load the data from second task from cloud storage to another BigQuery table using the schema provided loaded into the cloud storage which is uploaded from the src/schemas folder

template:
[JOB3](./team-repo/src/templates/job3.yaml)

1) Run the job. Similar as Job1

2) View the graph. Click on the link of the job and click on `Graph View`.

![JOB3](team-repo/airflow/images/job3.png "JOB 3")

3) View the Task Log

Click on any node in the graph and select View Log.

4) Check Output

After the job is successful we should be able to see two additional BigQuery tables loaded from the avro data in cloud storage both with schema and auto schema modes.

![JOB3 OUTPUT](team-repo/airflow/images/job3_output.png "JOB 3 OUTPUT")
![JOB3 OUTPUT](team-repo/airflow/images/job3_output2.png "JOB 3 OUTPUT")

### 4_dataflow_job_kubernetes

Airflow GCP operators provide different ways to orchestrate jobs within GCP. The `KubernetesPodOperator` can be used in place of the java or other operators to run generic jobs on Kubernetes. For this example we will use the GKE Pod Operator within your GKE cluster to extend execute java classes.

Before we can run this job we must build the dataflow job from the `dataflow` directory. This will create a container image with the dataflow job artifacts. 

From a terminal and the the root directory of this repo, run the command below. This will take several minutes.

In [ ]:
!gcloud builds submit team-repo/dataflow/kafka2avro/. --config=team-repo/dataflow/kafka2avro/cloudbuild.yaml

Once the image creation is completed, return to the airflow UI.

template:
[JOB4](./team-repo/src/templates/job4.yaml)

1) Run the job. Similar as Job1

2) View the graph . Click on the link of the job and click on Graph View

![JOB4](team-repo/airflow/images/job4.png "JOB 4")

3) View the Task Log

Click on any node in the graph and select View Log

4) Check Output
After the job is submitted we should be able to see the pod task in GKE and the dataflow job created by the pod.

![JOB4 OUTPUT](team-repo/airflow/images/job4_output.png "JOB 4 OUTPUT")
![JOB4 OUTPUT](team-repo/airflow/images/job4_output2.png "JOB 4 OUTPUT")

# Sample App to GKE

## Sample Application

This folder has a sample application with a server side api and a client side frondend deployed into gke environment of the team

## Server Side Spec

Python Flask application

Postgres db using CloudSQL via cloudsql proxy

GKE Deployment running unicorn app to expose flask api

## Client Side Spec

Vue Js application

GKE deployment with nginx 

## Deployment Steps

To Deploy we first need to create a postgres db and set some initial configuration such as create db, setting kubernetes secret in the namespace for cloudsql proxy and setting db username, password in
 kubernetes as secrets

Please set the enviroment variables:

```
PROJECT_ID -> project id of your gcp 
HOST_PROJECT_NAME -> project id of your gcp host project to create cloudsql in sharedvpc
NETWORK_NAME -> Shared VPC name of your host project
POSTGRES_PASSWORD -> postgress password to use to connect to db
```

Run the following script to create the db and secrets in kubernetes


```cloudsql.sh```


After the postgres db is created. We will need to seed data into the database for our api to expose

running the following cloudbuild step will create migration job in kubernetes to seed data into postgres

```gcloud builds submit . --config=cloudbuild-db-migration.yaml```

Then we can deploy the api & frontend into gke using the following step 

```gcloud builds submit . --config=cloudbuild.yaml```


# Datasynth

## About
### Input Files

The application uses several input files made by the team for the purposes of lookup:

- `long_lat.json` contains a list of latitudes and longitudes used to randomly assign to customers.
- `merchants_lat_long.json` contains a list of merchants GUIDs for each longitude and latitude location.
- `transactions_per_hour.json` contains a list of transactions per hour for each hour of the year.
- `merchant_details.json` contains a list of merchants (matches merchants_lat_long.json) with some detailed info: name, suburb, state, etc
- `recurring_merchants.json` contains a list of payees for recurring bill payments with such details as name, bpay biller code
- `cust_prod.json` contsains a list of subproduct codes for age between 18 to 68

### Customer, Account, and Attributes

Customers, Accounts, Customer to Account relationships, and Transaction specific Attributes will be generated.

This function will create the following files:

- `customers.json`
- `accounts.json`
- `customer_accounts.json`
- `attributes.json`
- `reccuring_payments.db`

### Cache

Customer available balances and transaction queue which are stored in-memory are dumped on disk into app/temp directory with the following frequency:
- every minute for kafka and stream
- every day in date range for batch

## Transactions Stream

Transactions will be produced at 1 second intervals and sent to `stdout` until the script is stopped.

## Transactions Kafka

Transactions will be produced at 1 second intervals and sent to kafka topic until the script is stopped.

## Transactions Batch

Transactions will be produced between 2 provided dates. A file per date will be created in the output directory.

This function will create the following files (example):

- `transactions_2018-01-01.json`
- `transactions_2018-01-02.json`

## Test
A test script for connecting to the datasynth stream is available in the `team-repo`. You can test the kafka connection using the following command:

In [ ]:
!team-repo/kafka/test.sh consumer synth-topic anz-cde-ic-ss --max-messages=5

## Payment Service Platform

Innovation Challenge Architecture 
Description: 

Payments Service Platform(PSP) API is meant for performing transactions with the accounts supplied. This involves sending the JSON payload over a POST call to the API which does the required validation of incoming message and once successful writes the payload to a Kafka topic. 

### PSP Service URLs 

TODO
`http://${PROJECT_ID}-pspapi.lab.kasna.cloud/psp/v1/submitPayment`

## Payload Validations: 
1. Date not in future 
2. Account is valid 
3. Customer related to account is active
4. Account has sufficient balance 

## PSP Service Contract 

### Input Message 

Sample Payload Request : SamplePayloadRequest.json 

### PSP API Responses: 

- Success 
```
{ 
    "reqId": "<<REQID>>", 
    "response": "PAYMENT_SUCCESS" 
}
```
    
- Future Date Error
```
{
    "reqId": "<<REQID>>",
    "response": "PAYMENT_FAILED - Future date not allowed"
}
```
- Amount exceeds available balance: 
```
{
    "reqId": "<<REQID>>", 
    "response": "PAYMENT_FAILED - Insufficient Amount" 
}
```

- Account not valid: 
```
{
    "reqId": "<<REQID>>", 
    "response": "PAYMENT_FAILED - Account ID is not Valid "
}
```

- Customer Inactive: 
```
{
    "reqId": "<<REQID>>", 
    "response": "PAYMENT_FAILED - Customer not present or inactive "
}
```

- Global Exception: 
```
{
    "reqId": "<REQID>>", 
    "response": "PAYMENT_FAILED - General Data Error"
}
```

### Kafka Topic Details: 
- Host : ${PROJECT_ID}-kafka.lab.kasna.internal
- Port : 9094 
- Topic Name : payment-topic 

# Theia IDE

## Theia IDE

Eclipse Theia is a web-based IDE modeled on VS Code. An instance of Theia is available in this jupyterhub environment and can use the team repo is already availble for use. To develop in this IDE do the following:
1. Navigate to the Launcher - if this is not open you can create a new Launcher using `File -> New Launcher`
2. Select the Theia icon

![Theia UI](team-repo/notebooks/images/open_theia.gif)

3. Once Theia is loaded select `File -> Open Workspace`
4. Navigate to `team-repo` and select open.

![Team Repo](team-repo/notebooks/images/workspace.gif)

Save the workspace to use during your next session.


# External DNS

You GKE (kubernetes) environment has the ability to update DNS addresses for both internal and external names. The domain names which will work are:
- `.lab.kasna.internal` Internal domain, used only for 10.x.x.x addresses
- `.lab.kasna.cloud` External domain, can be any external subdomain.

The GKE cluster uses external-dns service: https://github.com/kubernetes-incubator/external-dns

To add either domains, update you services manifest which is used in Kubernetes to include the following annotation:
```
metadata:
        annotations:
            external-dns.alpha.kubernetes.io/hostname: myservice.lab.kasna.internal.
```
For more information check the external-dns doco in the link above.

# Data Loss Prevention


The account numbers in the transactions in bigquery team dataset tables are currently tokenized using google cloud Data Loss Prevention
In this section we will query few transaction rows and call the dlp api to reidentify tokenized data
We can query the current DLP deidentification template by making an api request
First lets query BQ for data

In [1]:
%%bigquery
SELECT
acc_number
FROM `anz-uc-team-1.team_dataset.transactions*`
WHERE MOD(ABS(FARM_FINGERPRINT(txn_id)),10) != 0 AND txn_date BETWEEN '2019-07-02' AND '2019-07-03'

UsageError: Cell magic `%%bigquery` not found.


Now use the above account numbers in the following payload to query dlp api for actual account numbers

In [2]:
%%bash
gcloud auth list
curl -H "Authorization: Bearer $(gcloud auth application-default print-access-token)" \
-H "Content-Type: application/json" \
-X POST https://dlp.googleapis.com/v2/projects/anz-uc-team-1/content:reidentify \
-d @- << EOF
{
  "reidentifyConfig":{
    "recordTransformations":{
      "fieldTransformations":[
        {
          "primitiveTransformation":{
            "cryptoReplaceFfxFpeConfig":{
              "cryptoKey":{
                    "kmsWrapped": {
                      "wrappedKey": "CiQAWtiaI8FkG4exIuNZIhivv+fmr/fVU6qADcAidqutrMVgm+8SSADNHj8e+YISjf0IX5h0YSTxaC1D/GHXG/wVl67fjhjtZ8mS8jLMUV6MjvJuexkSEnWkE9ONYl4QnLWuy2JW7MiWQx3regxvJg==",
                      "cryptoKeyName": "projects/anz-uc-team-1/locations/global/keyRings/anz-uc-team-1-dlp-key/cryptoKeys/anz-uc-team-1-dlp-key"
                    }
              },
              "customAlphabet": "AC-1234567890"
            }
          },
          "fields":[
            {
              "name":"acc_number"
            }
          ]
        }
      ]
    }
  },
  "item":{
    "table":{
      "headers":[
        {
          "name":"acc_number"
        }
      ],
      "rows":[
        {
          "values":[
            {
              "stringValue":"9C9A-8A14C-61"
            }
          ]
        },
        {
          "values":[
            {
              "stringValue":"73057C777-1-91"
            }
          ]
        },
        {
          "values":[
            {
              "stringValue":"4841249CA33652"
            }
          ]
        }
        
      ]
    }
  }
}

EOF

                 Credentialed Accounts
ACTIVE  ACCOUNT
*       jupyter-sa@anz-uc-team-1.iam.gserviceaccount.com
{
  "item": {
    "table": {
      "headers": [
        {
          "name": "acc_number"
        }
      ],
      "rows": [
        {
          "values": [
            {
              "stringValue": "ACC-421950600"
            }
          ]
        },
        {
          "values": [
            {
              "stringValue": "ACC-2494704352"
            }
          ]
        },
        {
          "values": [
            {
              "stringValue": "ACC-2640076321"
            }
          ]
        }
      ]
    }
  },
  "overview": {
    "transformedBytes": "41",
    "transformationSummaries": [
      {
        "field": {
          "name": "acc_number"
        },
        "results": [
          {
            "count": "3",
            "code": "SUCCESS"
          }
        ],
        "fieldTransformations": [
          {
            "fields": [
              {
              


To set the active account, run:
    $ gcloud config set account `ACCOUNT`

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2902    0  1621  100  1281    772    610  0:00:02  0:00:02 --:--:--  1383
